<a href="https://colab.research.google.com/github/sajjadjim/Data-Minning-And-Machine-Learning/blob/main/Final_Machine_learning_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as stat
sample=pd.read_csv("/content/drive/MyDrive/Contest ML/obesity-level-prediction-fall-2024/sample_submission.csv")

test=pd.read_csv("/content/drive/MyDrive/Contest ML/obesity-level-prediction-fall-2024/test.csv")
print(test.head(10))
#show Data set
print(sample.head(10))
train=pd.read_csv("/content/drive/MyDrive/Contest ML/obesity-level-prediction-fall-2024/train.csv")

#show Data set
print(train.head(10))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     ID        Age  Gender    Height     Weight       CALC FAVC      FCVC  \
0  1111        NaN     NaN  1.708800  83.520113  Sometimes  yes  2.689577   
1  1112        NaN    Male  1.680991  79.752916        NaN  yes       NaN   
2  1113  27.349745    Male  1.835271  97.588260  Sometimes  yes  2.923433   
3  1114  18.000000    Male       NaN  86.565148  Sometimes  yes  3.000000   
4  1115  33.009285    Male  1.741192  84.773349         no  yes  2.000000   
5  1116        NaN    Male       NaN  86.080500  Sometimes  yes  2.882522   
6  1117  30.079371    Male       NaN  92.860254  Sometimes  yes  2.000000   
7  1118  25.035915    Male  1.763101  88.532032  Sometimes  yes  1.973499   
8  1119  18.198322  Female  1.543338  71.799982         no   no  3.000000   
9  1120  35.456326  Female       NaN  79.437921        NaN  yes  2.156065   

        NCP  SCC SMOKE 

In [ ]:
# prompt: train and test Null value sum

# Check for null values in the training dataset
print("Null values in the training dataset:\n", train.isnull().sum())

# Check for null values in the testing dataset
print("\nNull values in the testing dataset:\n", test.isnull().sum())

Null values in the training dataset:
 ID                                  0
Age                               309
Gender                            350
Height                            237
Weight                            357
CALC                               54
FAVC                              501
FCVC                                0
NCP                                 2
SCC                                 0
SMOKE                             333
CH2O                                0
family_history_with_overweight    120
FAF                                13
TUE                               215
CAEC                              320
MTRANS                            186
NObeyesdad                          0
dtype: int64

Null values in the testing dataset:
 ID                                  0
Age                               280
Gender                            339
Height                            220
Weight                            317
CALC                               68

In [ ]:

# Print data types of train and test datasets
print("\nData types of the training dataset:\n", train.dtypes)
print("\nData types of the testing dataset:\n", test.dtypes)


Data types of the training dataset:
 ID                                  int64
Age                               float64
Gender                             object
Height                            float64
Weight                            float64
CALC                               object
FAVC                               object
FCVC                              float64
NCP                               float64
SCC                                object
SMOKE                              object
CH2O                              float64
family_history_with_overweight     object
FAF                               float64
TUE                               float64
CAEC                               object
MTRANS                             object
NObeyesdad                         object
dtype: object

Data types of the testing dataset:
 ID                                  int64
Age                               float64
Gender                             object
Height                       

In [ ]:
# prompt: numerical_cols = train.select_dtypes(include=['float64', 'int64']).columns
# train[numerical_cols] = train[numerical_cols].fillna(train[numerical_cols].mean())  do this code better for fututre accuracy more find

# Identify numerical columns
numerical_cols = train.select_dtypes(include=['float64', 'int64']).columns

# Impute missing values using median for improved robustness
for col in numerical_cols:
    # Handle cases where all values might be NaN in a column
    if train[col].isnull().all():
        # If all values are NaN use 0 imputation, this is problem specific.
        train[col] = 0
    else:
        train[col] = train[col].fillna(train[col].median())


In [ ]:

categorical_cols = train.select_dtypes(include=['object']).columns

for col in categorical_cols:
    train[col] = train.groupby('NObeyesdad')[col].transform(lambda x: x.fillna(x.mode()[0]))

In [ ]:
train.isnull().sum()
#test.isnull().sum()

,0
ID,0
Age,0
Gender,0
Height,0
Weight,0
CALC,0
FAVC,0
FCVC,0
NCP,0
SCC,0


In [ ]:

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Initialize LabelEncoder
le = LabelEncoder()

# Identify categorical columns (excluding ID and target variable)
categorical_cols = train.select_dtypes(include=['object']).columns
categorical_cols = [col for col in categorical_cols if col not in ['ID', 'NObeyesdad']]

# Apply Label Encoding to categorical features
for col in categorical_cols:
    train[col] = le.fit_transform(train[col])

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), categorical_cols)], remainder='passthrough')

# Fit and transform the data using the column transformer
train_encoded = ct.fit_transform(train)


train.dtypes

,0
ID,int64
Age,float64
Gender,int64
Height,float64
Weight,float64
CALC,int64
FAVC,int64
FCVC,float64
NCP,float64
SCC,int64


In [ ]:
# prompt: from sklearn.model_selection import train_test_split
# X=train.drop(columns=['NObeyesdad'], axis=1)
# y=train['NObeyesdad']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)    do this better for my future model accuraccy find out the best

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier  # Example model, you can change this
from sklearn.metrics import accuracy_score


X = train.drop(columns=['NObeyesdad', 'ID'], axis=1)
y = train['NObeyesdad']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Define the model (RandomForestClassifier as an example, you can use any model)
model = RandomForestClassifier(random_state=42)


param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees
    'max_depth': [None, 10, 20],      # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4]     # Minimum samples required at each leaf node
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1) # Use all available cores
grid_search.fit(X_train, y_train)


best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print(f"Best Hyperparameters: {best_params}")


# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the best model on the test set: {accuracy}")


# Further analysis and refinement:

# 1. Try different models (e.g., GradientBoostingClassifier, SVC, XGBoost, etc.)
# 2. Explore a wider range of hyperparameters.
# 3. Consider feature engineering or selection techniques.
# 4. Use cross-validation more extensively (e.g., stratified k-fold).
# 5. Evaluate using a more comprehensive set of metrics (e.g., precision, recall, F1-score).

Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Accuracy of the best model on the test set: 0.7927927927927928


In [ ]:

from sklearn.model_selection import train_test_split
X=train.drop(columns=['NObeyesdad'], axis=1)
y=train['NObeyesdad']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8558558558558559


In [ ]:
rf=RandomForestClassifier(random_state=42)
rf.fit(X_train,y_train)
rf.score(X_test, y_test)

0.8558558558558559

In [ ]:

numerical_cols1 = test.select_dtypes(include=['float64', 'int64']).columns
test[numerical_cols1] = test[numerical_cols].fillna(test[numerical_cols].mean())

In [ ]:
categorical_cols1 = test.select_dtypes(include=['object']).columns
test[categorical_cols1] = test[categorical_cols1].fillna(test[categorical_cols1].mode().iloc[0])

test.isnull().sum()

,0
ID,0
Age,0
Gender,0
Height,0
Weight,0
CALC,0
FAVC,0
FCVC,0
NCP,0
SCC,0


In [ ]:
from sklearn.preprocessing import LabelEncoder
le1=LabelEncoder()
ctg1=test.drop(columns=['ID', 'Age', 'Height', 'Weight', 'NCP', 'CH2O', 'FAF', 'TUE', 'FCVC', 'NCP'])
for col in ctg1.columns:
  test[col]=le1.fit_transform(test[col])
test.dtypes

,0
ID,int64
Age,float64
Gender,int64
Height,float64
Weight,float64
CALC,int64
FAVC,int64
FCVC,float64
NCP,float64
SCC,int64


In [ ]:

# Now you can use test_encoded_df for prediction:
pred = rf.predict(test)
pred

array(['Overweight_Level_II', 'Overweight_Level_II',
       'Overweight_Level_II', ..., 'Overweight_Level_II',
       'Overweight_Level_II', 'Overweight_Level_II'], dtype=object)

In [ ]:
submission22=pd.DataFrame({'ID':test['ID'], 'NObeyesdad':pred})
submission22.to_csv('submission223.csv', index=False)
df=pd.read_csv('submission223.csv')